# Example: Single Agent

Install SUMO.

In [ ]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo

Install SMARTS.

In [ ]:
# Install SMARTS
%cd ~
!rm -rf /content/SMARTS
%pip freeze | grep -v 'pygobject\|python-apt\|screen-resolution-extra\|xkit' > uninstall.txt
%pip uninstall -y -r uninstall.txt

!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS
!cd /content/SMARTS && git checkout develop && pip install --force-reinstall .[camera-obs]
!echo -e "import sys\nsys.path.insert(0, '/content/SMARTS/')" | python

Build the scenario.

In [ ]:
# Build scenarios
!scl scenario build-all --clean /content/SMARTS/scenarios/figure_eight

Define the agent.

In [ ]:
from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

Define the environment loop.

In [ ]:
import gym

from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.core.utils.episodes import episodes
from smarts.env.wrappers.single_agent import SingleAgent

def main(scenarios, num_episodes, max_episode_steps=None):
    agent_spec = AgentSpec(
        interface=AgentInterface.from_type(
            AgentType.LanerWithSpeed, max_episode_steps=max_episode_steps,
        ),
        agent_builder=ChaseViaPointsAgent,
    )

    env = gym.make(
        "smarts.env:hiway-v0",
        scenarios=scenarios,
        agent_specs={"SingleAgent": agent_spec},
        headless=True,
        sumo_headless=True,
    )

    # Convert `env.step()` and `env.reset()` from multi-agent interface to
    # single-agent interface.
    env = SingleAgent(env=env)

    for episode in episodes(n=num_episodes):
        agent = agent_spec.build_agent()
        observation = env.reset()
        episode.record_scenario(env.scenario_log)

        done = False
        while not done:
            agent_action = agent.act(observation)
            observation, reward, done, info = env.step(agent_action)
            episode.record_step(observation, reward, done, info)

    env.close()

Run the example.

In [ ]:
# Enable offscreen render
# import os
# os.environ["PYOPENGL_PLATFORM"] = "egl"

# !pip install -r /content/SMARTS/requirements.txt
# %cd /content/SMARTS

main(
    scenarios=["/content/SMARTS/scenarios/figure_eight"],
    num_episodes=3,
    max_episode_steps=100,
)

Stop background processes

In [ ]:
%killbgscripts
!pkill -9 -f visdom
!pkill -9 -f smarts